In [1]:
# -----------------------------
# IMPORTS
# -----------------------------
import os
import re
import json
import numpy as np
from dotenv import load_dotenv
from mistralai import Mistral
import time
import random

# -----------------------------
# 0. Charger la clé depuis .env
# -----------------------------
load_dotenv()
api_key = os.getenv("MISTRAL_API_KEY")
client = Mistral(api_key=api_key)

# -----------------------------
# 1. Fonctions utilitaires
# -----------------------------
def show_tree(root):
    """Renvoie l'arborescence du dossier sous forme de texte."""
    lines = []
    for dirpath, dirnames, filenames in os.walk(root):
        level = dirpath.replace(root, "").count(os.sep)
        indent = " " * 4 * level
        lines.append(f"{indent}{os.path.basename(dirpath)}/")
        subindent = " " * 4 * (level + 1)
        for f in filenames:
            lines.append(f"{subindent}{f}")
    return "\n".join(lines)

def classify_files(root, code_ext=None, doc_ext=None):
    """Classe les fichiers du dossier en codes, docs et autres."""
    code_ext = code_ext or {".py", ".java", ".cpp", ".c", ".js", ".ts", ".ipynb"}
    doc_ext  = doc_ext  or {".txt", ".pdf", ".doc", ".docx", ".xls", ".xlsx", ".ppt", ".pptx", ".md", ".csv"}
    codes, docs, others = [], [], []
    for dirpath, _, filenames in os.walk(root):
        for f in filenames:
            ext = os.path.splitext(f)[1].lower()
            full_path = os.path.join(dirpath, f)
            if ext in code_ext:
                codes.append(full_path)
            elif ext in doc_ext:
                docs.append(full_path)
            else:
                others.append(full_path)
    return codes, docs, others

def read_code(file_path):
    """Lit un fichier Python ou notebook et renvoie son code."""
    ext = os.path.splitext(file_path)[1].lower()
    if ext == ".ipynb":
        with open(file_path, "r", encoding="utf-8") as f:
            nb = json.load(f)
            content = []
            for cell in nb.get("cells", []):
                if cell.get("cell_type") == "code":
                    content.append("".join(cell.get("source", [])))
            return "\n".join(content)
    else:
        with open(file_path, "r", encoding="utf-8") as f:
            return f.read()

def extract_imports(code):
    """Retourne un set de toutes les bibliothèques importées dans le code."""
    imports = set()
    for line in code.splitlines():
        line = line.strip()
        if line.startswith("import "):
            imports.add(line.replace("import ", "").split()[0])
        elif line.startswith("from "):
            imports.add(line.split()[1])
    return imports

def split_functions(code):
    """Découpe le code en chunks par fonction."""
    pattern = r"(def [\w_]+\s*\(.*?\):(?:\n(?:\s+.+))*)"
    funcs = re.findall(pattern, code, re.DOTALL)
    return funcs if funcs else [code]

def describe_file(file_name, code, max_tokens=1000):
    """Génère un résumé rapide d’un fichier code via le LLM."""
    chunks = split_functions(code)
    combined = "\n\n".join(chunks)
    prompt = f"""
Tu es un expert Python. Voici le code complet du fichier :

---
{combined}
---

Fais un résumé Markdown rapide avec :
# Résumé global du fichier
# Bibliothèques utilisées
# Description des fonctions
"""
    response = client.chat.complete(
        model="codestral-2508",
        messages=[{"role": "user", "content": prompt}],
        temperature=0.2,
        max_tokens=max_tokens
    )
    return response.choices[0].message.content

def generate_readme(folder_path, codes, docs, file_summaries, file_contents, output_file):
    """Génère un README.md complet du dossier."""
    # Arborescence
    readme = "# Documentation Automatique du Dossier\n\n"
    readme += "## Arborescence du dossier\n"
    readme += show_tree(folder_path)
    readme += "\n\n---\n\n"

    # Fichiers codes / docs
    readme += "## Fichiers Codes\n"
    for f in codes:
        readme += f"- {f}\n"
    readme += "\n"
    readme += "## Documents\n"
    for f in docs:
        readme += f"- {f}\n"
    readme += "\n"

    # Bibliothèques utilisées
    all_imports = set()
    for f in file_summaries.keys():
        code = file_contents[f]
        all_imports.update(extract_imports(code))
    if all_imports:
        readme += "## Bibliothèques nécessaires pour le projet\n"
        for lib in sorted(all_imports):
            readme += f"- {lib}\n"
        readme += "\n"

    # Résumé rapide par fichier
    readme += "## Résumé rapide des fichiers code\n"
    for f, summary in file_summaries.items():
        readme += f"### {f}\n"
        readme += summary
        readme += "\n\n---\n\n"

    # Sauvegarde
    with open(output_file, "w", encoding="utf-8") as f:
        f.write(readme)
    print(f" README.md généré ici : {output_file}")



In [2]:
# -----------------------------
# 1. Demande à l'utilisateur le dossier
# -----------------------------
folder_path = input("Chemin du dossier à analyser : ").strip()
if not os.path.exists(folder_path):
    raise ValueError("Le dossier n'existe pas !")

output_file = os.path.join(folder_path, "README.md")
if os.path.exists(output_file):
    os.remove(output_file)

# -----------------------------
# 2. Classer les fichiers
# -----------------------------
codes, docs, others = classify_files(folder_path)

# -----------------------------
# 3. Lire le code et extraire résumé
# -----------------------------
file_contents = {}
file_summaries = {}
for f in codes:
    code = read_code(f)
    file_contents[f] = code
    print(f"Analyse du fichier {f} ...")
    file_summaries[f] = describe_file(f, code)

# -----------------------------
# 4. Générer le README.md
# -----------------------------
generate_readme(folder_path, codes, docs, file_summaries, file_contents, output_file)


Analyse du fichier D:\33611\Documents\MASTER\M2\PROJET_README\creating-a-README-for-a-codebase\TESTS\TEST2\code3.py ...
Analyse du fichier D:\33611\Documents\MASTER\M2\PROJET_README\creating-a-README-for-a-codebase\TESTS\TEST2\DOSSIER 1\code1.py ...
Analyse du fichier D:\33611\Documents\MASTER\M2\PROJET_README\creating-a-README-for-a-codebase\TESTS\TEST2\DOSSIER 1\code5.ipynb ...
Analyse du fichier D:\33611\Documents\MASTER\M2\PROJET_README\creating-a-README-for-a-codebase\TESTS\TEST2\DOSSIER 2\code2.py ...
Analyse du fichier D:\33611\Documents\MASTER\M2\PROJET_README\creating-a-README-for-a-codebase\TESTS\TEST2\DOSSIER 2\code4.ipynb ...
 README.md généré ici : D:\33611\Documents\MASTER\M2\PROJET_README\creating-a-README-for-a-codebase\TESTS\TEST2\README.md
